<a href="https://colab.research.google.com/github/jwbarbona/labo2025v/blob/main/LSTM_avanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Primero seleccionar Runtime T4 GPU**

*La utilizaremos para obtener tiempos razonables de ejecución.*





In [ ]:
# Instalar TensorFlow si no está disponible (necesario en entornos nuevos de Colab)
!pip install tensorflow -q

# Modelos Encoder-Decoder para Traducción Automática

En este cuaderno se explora la implementación de una arquitectura encoder-decoder utilizando redes LSTM para tareas de traducción automática de inglés a español. Se emplea un corpus paralelo del Proyecto Tatoeba.

## Objetivos de Aprendizaje
- Comprender la arquitectura encoder-decoder para tareas secuencia a secuencia.
- Preprocesar datos de texto paralelos para traducción automática.
- Implementar un encoder y decoder basado en LSTM.
- Entrenar el modelo utilizando teacher forcing.
- Realizar inferencia con el modelo entrenado.
- Evaluar la calidad de la traducción de manera cualitativa.

**Nota:** Este es un ejemplo simplificado con fines didácticos. En la práctica, se utilizan técnicas más avanzadas como mecanismos de atención y búsqueda en haz para un mejor rendimiento.

**Conceptos Clave a Recordar:**
- **Encoder-Decoder:** El encoder comprime la entrada en un vector de contexto; el decoder genera la salida a partir de este vector.
- **Teacher Forcing:** Técnica de entrenamiento donde se proporciona la salida real anterior en lugar de la predicha.
- **Inferencia:** Proceso autoregresivo para generar traducciones token por token.

**Pregunta para Reflexión:** ¿Por qué la arquitectura encoder-decoder es adecuada para tareas como la traducción, donde las longitudes de entrada y salida pueden diferir?

## Explicación Detallada del Problema

La traducción automática consiste en convertir texto de un idioma a otro utilizando una máquina, sin intervención humana directa. Imagínese tener una conversación con alguien que habla otro idioma: en lugar de buscar un diccionario o un traductor humano, un sistema podría traducir al instante las palabras para que ambos se entiendan.

En términos simples, este problema se parece a convertir una receta en inglés a español, manteniendo el significado original. Es útil en el mundo real para viajar, negocios internacionales o acceder a información en idiomas extranjeros, facilitando la comunicación global. Aquí, se utiliza un conjunto de datos con oraciones en inglés y sus equivalentes en español para entrenar un modelo que 'aprenda' a traducir, de forma accesible y sin necesidad de conocimientos técnicos profundos para captar la idea básica.

## 1. Importación de Bibliotecas

**Explicación:** Estas bibliotecas son esenciales para el preprocesamiento, modelado y visualización. TensorFlow y Keras facilitan la construcción de modelos recurrentes, mientras que NumPy y Pandas ayudan en el manejo de datos.

In [ ]:
# Importación de bibliotecas para expresiones regulares y datos numéricos
import re  # Para limpieza de texto mediante expresiones regulares
import numpy as np  # Para operaciones con arreglos y matrices
import pandas as pd  # Para manipulación de datos tabulares
import os  # Para chequeos de archivos

# Bibliotecas para aprendizaje profundo
import tensorflow as tf  # Framework principal
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense  # Capas del modelo
from tensorflow.keras.models import Model  # Para definir modelos
from tensorflow.keras.preprocessing.text import Tokenizer  # Para tokenización
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Para relleno de secuencias
from sklearn.model_selection import train_test_split  # Para división de datos

# Bibliotecas para visualización
import matplotlib.pyplot as plt  # Para gráficos
import random  # Para selección aleatoria

In [ ]:
# Verificar disponibilidad de GPU para aceleración
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("GPU detectada: El entrenamiento se acelerará automáticamente.")
else:
    print("No se detectó GPU: El entrenamiento usará CPU (más lento). Cambie el tipo de runtime en Colab a GPU para mejorar el rendimiento.")

## 2. Carga y Exploración del Conjunto de Datos

Se utiliza el corpus paralelo inglés-español de http://www.manythings.org/anki/

**Explicación:** La carga de datos paralelos es fundamental en traducción automática, ya que cada oración en un idioma corresponde a su equivalente en el otro.

In [ ]:
# Chequea si spa-eng.zip ya fue descargado
if not os.path.exists("spa-eng.zip"):
    # Descarga y descompresión del archivo
    !wget http://www.manythings.org/anki/spa-eng.zip
    !unzip spa-eng.zip -d spa-eng

In [ ]:
# Ruta al archivo de datos
ruta_datos = 'spa-eng/spa.txt'
# Lectura del archivo
with open(ruta_datos, 'r', encoding='utf-8') as f:
    líneas = f.read().split('\n')

## Recorrido por el Conjunto de Datos

El conjunto de datos Tatoeba consta de pares de oraciones en inglés y español, con aproximadamente 120.000 ejemplos. Cada línea del archivo contiene una oración en inglés, su traducción al español y metadatos separados por tabuladores.

**Estructura:**
- Líneas del archivo: Oración en inglés \t Oración en español \t Atribución.
- Después de carga: Listas de textos en inglés y español.

**Contenidos:** Las oraciones cubren conversaciones cotidianas, frases simples y complejas, ideales para aprender patrones lingüísticos.

**Relación con la Solución:** Este corpus paralelo permite que el modelo encoder-decoder aprenda a mapear secuencias de un idioma a otro. El preprocesamiento (limpieza, tokenización) prepara los datos para que el modelo capture dependencias secuenciales y genere traducciones coherentes.

**Ejemplos Adicionales:**
A continuación se muestran más pares de oraciones para ilustrar el contenido.

In [ ]:
# Impresión de más ejemplos
print("Ejemplo de línea 1:", líneas[0])
print("Ejemplo de línea 2:", líneas[1])
print("Ejemplo de línea 3:", líneas[2])

# Número total de líneas
print("\nNúmero total de pares:", len(líneas) - 1)  # Menos uno por posible línea vacía

### Muestra de Datos

**Explicación:** Visualizar muestras ayuda a verificar la integridad del corpus y entender su estructura.

In [ ]:
# Lista para pares de muestras
pares_muestras = []
# Procesamiento de las primeras 10 líneas
for línea in líneas[:10]:
    if '\t' in línea:
        ing, esp, _ = línea.split('\t')  # Separación por tabulador
        pares_muestras.append((ing, esp))

# Creación de DataFrame para visualización
df_muestras = pd.DataFrame(pares_muestras, columns=['Inglés', 'Español'])
print("Una Mirada al Corpus Paralelo:")
display(df_muestras)

## 3. Preprocesamiento de Datos

### Función de Limpieza de Texto

**Explicación:** La limpieza elimina ruido como puntuación mal colocada y caracteres no deseados, facilitando la tokenización.

In [ ]:
# Función para limpiar texto
def limpiar_texto(t):
    t = re.sub(r"([?.!,¿])", r" \1 ", t)  # Separar puntuación
    t = re.sub(r'[" "]+', " ", t)  # Normalizar espacios
    t = re.sub(r"[^a-zA-Z?.!,¿¿]+", " ", t)  # Eliminar caracteres no alfabéticos ni puntuación
    t = t.strip()  # Eliminar espacios iniciales/finales
    return t

Se agrega un token `start` al inicio y `end` al final de cada oración objetivo (español). Estos tokens actúan como señales explícitas, transformando la tarea de predecir el objetivo dado la fuente, en un proceso estructurado de predecir el siguiente token dado la fuente y todos los tokens generados previamente.

**Pregunta para Reflexión:** ¿Cómo afectan estos tokens al proceso de inferencia?

In [ ]:
# Función para crear el conjunto de datos
def crear_conjunto_datos(ruta, num_ejemplos):
    líneas = open(ruta, encoding='UTF-8').read().strip().split('\n')  # Lectura y división
    pares_palabras = [[limpiar_texto(s) for s in l.split('\t')[:2]] for l in líneas[:num_ejemplos]]  # Limpieza

    # Agregar tokens de inicio y fin al idioma objetivo
    for par in pares_palabras:
        par[1] = 'start ' + par[1] + ' end'

    return zip(*pares_palabras)

# Uso de un subconjunto para entrenamiento rápido
num_ejemplos = 10000  # Reducido para ejecución rápida en clase; en casa usar 30000 o más
texto_entrada, texto_objetivo = crear_conjunto_datos(ruta_datos, num_ejemplos)

print("Par de Muestra Preprocesado:")
print("Entrada (Inglés):", texto_entrada[0])
print("Objetivo (Español):", texto_objetivo[0])

## 4. Tokenización

**Explicación:** La tokenización convierte texto en secuencias numéricas, esenciales para el procesamiento por redes neuronales.

In [ ]:
# Tokenizador para entrada
tokenizador_entrada = Tokenizer()
tokenizador_entrada.fit_on_texts(texto_entrada)  # Ajuste al texto
tensor_entrada = tokenizador_entrada.texts_to_sequences(texto_entrada)  # Conversión a secuencias

# Tokenizador para objetivo
tokenizador_objetivo = Tokenizer()
tokenizador_objetivo.fit_on_texts(texto_objetivo)
tensor_objetivo = tokenizador_objetivo.texts_to_sequences(texto_objetivo)

# Tamaños de vocabulario
tamaño_vocab_entrada = len(tokenizador_entrada.word_index) + 1
tamaño_vocab_objetivo = len(tokenizador_objetivo.word_index) + 1

print(f"Tamaño de vocabulario de entrada: {tamaño_vocab_entrada}")
print(f"Tamaño de vocabulario de objetivo: {tamaño_vocab_objetivo}")

## 5. Relleno de Secuencias

**Explicación:** El relleno asegura que todas las secuencias tengan la misma longitud para procesar en lotes.

In [ ]:
# Función para rellenar secuencias
def rellenar_secuencias(tensor):
    return pad_sequences(tensor, padding='post')  # Relleno al final

tensor_entrada_rellenado = rellenar_secuencias(tensor_entrada)
tensor_objetivo_rellenado = rellenar_secuencias(tensor_objetivo)

longitud_máx_entrada = tensor_entrada_rellenado.shape[1]
longitud_máx_objetivo = tensor_objetivo_rellenado.shape[1]

print(f"Longitud máxima de secuencias de entrada: {longitud_máx_entrada}")
print(f"Longitud máxima de secuencias de objetivo: {longitud_máx_objetivo}")

## 6. Preparación de Datos para Teacher Forcing

**Explicación:** Teacher forcing acelera el entrenamiento al proporcionar la salida correcta en cada paso.

In [ ]:
# Datos de entrada y objetivo para decoder
datos_entrada_decoder = np.zeros((len(tensor_objetivo), longitud_máx_objetivo))
datos_objetivo_decoder = np.zeros((len(tensor_objetivo), longitud_máx_objetivo))

for i, obj_t in enumerate(tensor_objetivo):
    datos_entrada_decoder[i, 0:len(obj_t)] = obj_t  # Secuencia completa
    datos_objetivo_decoder[i, 0:len(obj_t)-1] = obj_t[1:]  # Secuencia desplazada
    datos_objetivo_decoder[i, len(obj_t)-1:] = 0  # Relleno con ceros

# División en conjuntos de entrenamiento y validación
entrada_encoder_entren, entrada_encoder_val, entrada_decoder_entren, entrada_decoder_val, objetivo_decoder_entren, objetivo_decoder_val = train_test_split(
    tensor_entrada_rellenado, datos_entrada_decoder, datos_objetivo_decoder, test_size=0.2, random_state=42)

## Demostración Rápida de LSTM para Predicción de Series Temporales

**Explicación:** Las LSTM no solo sirven para traducción, sino también para series temporales, donde capturan patrones en datos secuenciales como temperaturas o precios. Aquí se presenta un ejemplo simple con una serie sinusoidal para ilustrar su aplicación, si el tiempo en clase lo permite.

In [ ]:
# Generación de datos sintéticos: onda sinusoidal
time_steps = np.linspace(0, 10 * np.pi, 1000)
data = np.sin(time_steps)

# Preparación de secuencias para LSTM (ventana de 50 pasos para predecir el siguiente)
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:i + seq_length])
        ys.append(data[i + seq_length])
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 50
X, y = create_sequences(data, SEQ_LENGTH)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Forma para LSTM

# División en entrenamiento y prueba
split = int(0.8 * len(X))
X_train_ts, X_test_ts = X[:split], X[split:]
y_train_ts, y_test_ts = y[:split], y[split:]

# Construcción del modelo LSTM simple para series temporales
model_ts = tf.keras.Sequential([
    Input(shape=(SEQ_LENGTH, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])
model_ts.compile(optimizer='adam', loss='mse')

# Entrenamiento
history_ts = model_ts.fit(X_train_ts, y_train_ts, epochs=2, validation_data=(X_test_ts, y_test_ts), verbose=1)  # Limitado a 2 épocas para ejecución rápida

# Predicciones
y_pred_ts = model_ts.predict(X_test_ts)

# Visualización
plt.figure(figsize=(10, 5))
plt.plot(y_test_ts, label='Valor Real')
plt.plot(y_pred_ts, label='Predicción LSTM')
plt.title('Predicción de Serie Temporal con LSTM')
plt.xlabel('Muestras de Prueba')
plt.ylabel('Valor')
plt.legend()
plt.show()

print('**Nota:** Esta demostración muestra cómo las LSTM capturan dependencias temporales, similar a cómo procesan secuencias en traducción.')

## 7. Definición de Hiperparámetros del Modelo

In [ ]:
# Hiperparámetros del modelo
dimensión_incrustacion = 256
unidades_ocultas = 1024

Ahora, se construirá el modelo pieza por pieza, comenzando con el encoder.

**Explicación:** El encoder resume la entrada en estados ocultos, que el decoder usa para generar la salida.

In [ ]:
# 1. El Encoder
entradas_encoder = Input(shape=(None,), name='entradas_encoder')  # Entrada variable
capa_incrustacion_encoder = Embedding(tamaño_vocab_entrada, dimensión_incrustacion, name='incrustacion_encoder')
incrustacion_encoder = capa_incrustacion_encoder(entradas_encoder)

lstm_encoder = LSTM(unidades_ocultas, return_state=True, name='lstm_encoder')
salidas_encoder, estado_h, estado_c = lstm_encoder(incrustacion_encoder)
estados_encoder = [estado_h, estado_c]

# 2. El Decoder
entradas_decoder = Input(shape=(None,), name='entradas_decoder')
capa_incrustacion_decoder = Embedding(tamaño_vocab_objetivo, dimensión_incrustacion, name='incrustacion_decoder')
incrustacion_decoder = capa_incrustacion_decoder(entradas_decoder)

lstm_decoder = LSTM(unidades_ocultas, return_sequences=True, return_state=True, name='lstm_decoder')
salidas_decoder, _, _ = lstm_decoder(incrustacion_decoder, initial_state=estados_encoder)

densa_decoder = Dense(tamaño_vocab_objetivo, activation='softmax', name='densa_decoder')
salidas_decoder = densa_decoder(salidas_decoder)

# 3. Ensamblaje del Modelo de Entrenamiento
modelo_entrenamiento = Model([entradas_encoder, entradas_decoder], salidas_decoder)

# Visualización de la arquitectura
modelo_entrenamiento.summary()
tf.keras.utils.plot_model(modelo_entrenamiento, to_file='modelo_entrenamiento.png', show_shapes=True)

## 8. Compilación del Modelo

**Explicación:** La compilación define la función de pérdida y el optimizador.

In [ ]:
# Compilación del modelo
modelo_entrenamiento.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 9. Entrenamiento del Modelo

**Explicación:** Durante el entrenamiento, se utiliza teacher forcing para proporcionar la salida correcta en cada paso.

In [ ]:
# Parámetros de entrenamiento
épocas = 2  # Limitado a 2 para ejecución rápida en clase; en casa usar 20 o más
tamaño_lote = 64

historial = modelo_entrenamiento.fit(
    [entrada_encoder_entren, entrada_decoder_entren],
    objetivo_decoder_entren[:, :, np.newaxis],  # Dimensión para loss
    batch_size=tamaño_lote,
    epochs=épocas,
    validation_data=([entrada_encoder_val, entrada_decoder_val], objetivo_decoder_val[:, :, np.newaxis])
)

## 10. Gráfico del Historial de Entrenamiento

In [ ]:
# Función para graficar historial
def graficar_historial_entrenamiento(historial):
    # Gráfico de precisión
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(historial.history['accuracy'], 'bo-', label='Entrenamiento')
    plt.plot(historial.history['val_accuracy'], 'ro-', label='Validación')
    plt.title('Precisión del Modelo')
    plt.ylabel('Precisión')
    plt.xlabel('Época')
    plt.legend(loc='upper left')
    plt.grid(True)

    # Gráfico de pérdida
    plt.subplot(1, 2, 2)
    plt.plot(historial.history['loss'], 'bo-', label='Entrenamiento')
    plt.plot(historial.history['val_loss'], 'ro-', label='Validación')
    plt.title('Pérdida del Modelo')
    plt.ylabel('Pérdida')
    plt.xlabel('Época')
    plt.legend(loc='upper left')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

graficar_historial_entrenamiento(historial)

## 11. Modelos para Inferencia

**Explicación:** Durante la inferencia, el decoder genera token por token, utilizando sus predicciones previas.

In [ ]:
# Modelo encoder para inferencia
modelo_encoder = Model(entradas_encoder, estados_encoder)

# Entradas de estados para decoder
entradas_estados_decoder = [Input(shape=(unidades_ocultas,)), Input(shape=(unidades_ocultas,))]
entrada_simple_decoder = Input(shape=(1,))
incrustacion_simple_decoder = capa_incrustacion_decoder(entrada_simple_decoder)

# Reutilización de LSTM decoder
salidas_decoder, estado_h, estado_c = lstm_decoder(incrustacion_simple_decoder, initial_state=entradas_estados_decoder)
estados_decoder = [estado_h, estado_c]

# Reutilización de capa densa
salidas_decoder = densa_decoder(salidas_decoder)

# Modelo decoder para inferencia
modelo_decoder = Model([entrada_simple_decoder] + entradas_estados_decoder, [salidas_decoder] + estados_decoder)

modelo_encoder.summary()
modelo_decoder.summary()

## 12. Función para Decodificar Secuencia

In [ ]:
# Índices inversos para decodificación
índice_char_entrada_inverso = dict((i, palabra) for palabra, i in tokenizador_entrada.word_index.items())
índice_char_objetivo_inverso = dict((i, palabra) for palabra, i in tokenizador_objetivo.word_index.items())

# Función de decodificación
def decodificar_secuencia(sec_entrada):
    # Obtención de estados del encoder
    valores_estados = modelo_encoder.predict(sec_entrada, verbose=0)
    # Secuencia objetivo inicial con start
    sec_objetivo = np.zeros((1, 1))
    sec_objetivo[0, 0] = tokenizador_objetivo.word_index['start']

    condición_paro = False
    oración_decodificada = ''

    # Bucle de generación autoregresiva
    while not condición_paro:
        tokens_salida, h, c = modelo_decoder.predict([sec_objetivo] + valores_estados, verbose=0)
        índice_token_muestreado = np.argmax(tokens_salida[0, -1, :])
        char_muestreada = índice_char_objetivo_inverso.get(índice_token_muestreado, '<des>')

        # Condición de salida
        if (char_muestreada == 'end' or len(oración_decodificada.split()) > longitud_máx_objetivo):
            condición_paro = True
        else:
            oración_decodificada += ' ' + char_muestreada

        # Actualización de secuencia objetivo
        sec_objetivo = np.zeros((1, 1))
        sec_objetivo[0, 0] = índice_token_muestreado

        # Actualización de estados
        valores_estados = [h, c]

    return oración_decodificada.strip()

## 13. Prueba de Traducciones

In [ ]:
# Selección de índices aleatorios
índices_muestra = np.random.choice(range(len(entrada_encoder_val)), 5, replace=False)
resultados = []

for i in índices_muestra:
    sec_entrada = entrada_encoder_val[i:i+1]
    traducción_predicha = decodificar_secuencia(sec_entrada)

    # Conversión de secuencias a texto
    oración_entrada = ' '.join([índice_char_entrada_inverso.get(i, '') for i in sec_entrada[0] if i != 0])

    tokens_oración_objetivo = [índice_char_objetivo_inverso.get(i, '') for i in entrada_decoder_val[i] if i != 0 and i not in [tokenizador_objetivo.word_index.get('start', 0), tokenizador_objetivo.word_index.get('end', 0)]]
    oración_objetivo = ' '.join(tokens_oración_objetivo).strip()

    resultados.append((oración_entrada, oración_objetivo, traducción_predicha))

# Visualización en DataFrame
df_resultados = pd.DataFrame(resultados, columns=['Entrada (Inglés)', 'Objetivo (Español)', 'Predicho (Español)'])
display(df_resultados)

## Conclusión

Se demostró un modelo encoder-decoder básico para traducción automática. Para mejorar el rendimiento, se recomienda agregar mecanismos de atención o utilizar arquitecturas de transformadores.

**Actividad Sugerida:** Implementar un mecanismo de atención simple y compare los resultados cualitativos.